In [ ]:
import os
from tinytag import TinyTag, TinyTagException
from sklearn.neighbors import NearestNeighbors
from collections import defaultdict
from keras.models import load_model
import librosa
from collections import Counter
import multiprocessing
from tqdm import tqdm
from keras.models import Model
import numpy as np
import sounddevice as sd

In [ ]:
MUSIC_ROOT = '/home/af/Dokumenter/Personal/Music/Mors Musik/'
mp3s = []
for root, subdirs, files in os.walk(MUSIC_ROOT):
    for fn in files:
        if fn.endswith('.mp3'):
            mp3s.append(os.path.join(root, fn))
len(mp3s)

In [ ]:
TO_SKIP = {'Podcast', 'Books & Spoken'}

def process_mp3(path):
    try:
        tag = TinyTag.get(path)
        if tag.genre in TO_SKIP:
            return None
    except TinyTagException:
        print('error')
        return None
    signal, sr = librosa.load(path, res_type='kaiser_fast', offset=30, duration=30)
    try:
        melspec = librosa.feature.melspectrogram(signal, sr=sr).T[:1280,]
        if len(melspec) != 1280:
            return None
    except ValueError:
        return None
    return {'path': path,
            'melspecs': np.asarray(np.split(melspec, 10)),
            'tag': tag}

songs = [process_mp3(path) for path in tqdm(mp3s[:1000])]
songs = [song for song in songs if song]

In [ ]:
inputs = []
for song in songs:
    inputs.extend(song['melspecs'])
inputs = np.array(inputs)
inputs.shape

In [ ]:
cnn_model = load_model('../data/zoo/15/song_classify.h5')
vectorize_model = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-4].output)
vectors = vectorize_model.predict(inputs)
vectors.shape

In [ ]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(vectors)

def most_similar_songs(song_idx):
    distances, indices = nbrs.kneighbors(vectors[song_idx * 10: song_idx * 10 + 10])
    c = Counter()
    for row in indices:
        for idx in row[1:]:
            c[idx // 10] += 1
    return c.most_common()

In [ ]:
song_idx = 7
print(songs[song_idx]['path'])

print('---')
for idx, score in most_similar_songs(song_idx)[:5]:
    print(songs[idx]['path'], score)
print('')

In [ ]:
duration = 30  # seconds
fs = 22050
myrecording = sd.rec(int(duration * fs), samplerate=fs, channels=1)

In [ ]:
myrecording.shape

In [ ]:
sd.play(myrecording, samplerate=fs)

In [ ]:
myrecording.min()

In [ ]:
signal, sr = librosa.load('/home/af/Dokumenter/Personal/Music/Mors Musik/Diverse/Dansk filmmusik og danske sange Vol.1/'
                          '01 - I Danmark er jeg født.mp3', res_type='kaiser_fast', offset=0, duration=30)

melspec = librosa.feature.melspectrogram(signal, sr=sr).T[:1280,]
melspecs = np.asarray(np.split(melspec, 10))
melspecs.shape

In [ ]:
recorded_vectors = vectorize_model.predict(melspecs)

In [ ]:
distances, indices = nbrs.kneighbors(recorded_vectors)
c = Counter()
for row in indices:
    for idx in row[1:]:
        c[idx // 10] += 1
for idx, _ in c.most_common():
    print(songs[idx]['path'])


In [ ]:
signal, sr = librosa.load('/home/af/Dokumenter/Personal/Music/Mors Musik/Diverse/Dansk filmmusik og danske sange Vol.1/'
                          '01 - I Danmark er jeg født.mp3', res_type='kaiser_fast', offset=0, duration=30)

In [ ]:
sd.play(signal.flatten(), samplerate=sr)